# Лабораторная работа 5. Рекомендации по формализации и расчетам характеристик подсистем КИС в виде разомкнутых или замкнутых стохастических моделей
## Задание 1

Для одноканальной системы массового обслуживания с ограничением на длину
очереди $m$ составьте дифференциальные уравнения для вероятностей нахождения
в заданных состояниях в зависимости от времени. Найдите эти вероятности при
определенном в соответствии с вариантом значении $t$, а также при $t
\xrightarrow{} 0$. Канал иногда может выходить из строя. Заявка, которая
обслуживается в момент отказа канала ставится в очередь, если там есть места,
в противном случае она покидает систему необслуженной. Входящий поток, поток
обслуживания, поток отказов и поток восстановления простейшие
с соответствующими интенсивностями $\lambda, \mu, \nu, \gamma$. Количество
клиентов, от которых могут поступать заявки на обслуживание $k$. Начальные
условия $P_0(0) = 1$.

In [14]:
Variant <- 5
set.seed(Variant)
m <- sample(c(4:18), 1)

mu <- runif(1)

lambda <- runif(1)
if (lambda > mu) {
    current <- lambda
    lambda <- mu
    mu <- current
}

gamma <- runif(1)

nu <- runif(1)

if (gamma < nu) {
    current <- nu
    nu <- gamma
    gamma <- current
}

if (sample(c(0:1), 1)) {
    k <- sample(c(4:7), 1)
} else {
    k <- "inf"
}
t <- runif(1)
View(data.frame(lambda, mu, nu, gamma, k, m, t))

lambda,mu,nu,gamma,k,m,t
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<dbl>
0.6852186,0.9168758,0.1046501,0.2843995,inf,5,0.52796


Введем следующие состояния:
- $S_0$ - нет выполняемых задач, СМО готова выполнять внешние задачи;
- $S_1$ - СМО занята выполнением внешней задачи, очереди нет;
- $S_2$ - СМО занята выполнением внешней задачи, в очереди одна задача;
- ...
- $S_m$ - СМО занята выполнением внешней задачи, в очереди $m$ задач;
- $S_{m+1}$ - СМО вышла из строя, в очереди нет задач;
- $S_{m+2}$ - СМО вышла из строя, в очереди одна задача;
- ...
- $S_{2m}$ - СМО вышла из строя, в очереди m задач.

Тогда граф состояний будет выглядеть:
![graph](./State_graph.png)
В бирюзовом кластере находятся состояния, когда СМО в рабочем состоянии.
В зеленом - когда вышла из строя.

По графу составим уравнения Колмогорова:
$$
\begin{cases}
\frac{dP_0(t)}{dt} = -(\lambda + \nu)P_0(t) + \mu\cdot P_1(t) + \gamma \cdot P_6(t) \\
\frac{dP_1(t)}{dt} = -(\lambda + \nu + \mu)P_1(t) + \lambda \cdot P_0(t) + \mu \cdot P_2(t) + \gamma \cdot P_7(t) \\
\frac{dP_2(t)}{dt} = -(\lambda + \nu + \mu)P_2(t) + \lambda \cdot P_1(t) + \mu \cdot P_3(t) + \gamma \cdot P_8(t) \\
\frac{dP_3(t)}{dt} = -(\lambda + \nu + \mu)P_3(t) + \lambda \cdot P_2(t) + \mu \cdot P_4(t) + \gamma \cdot P_9(t) \\
\frac{dP_4(t)}{dt} = -(\lambda + \nu + \mu)P_4(t) + \lambda \cdot P_3(t) + \mu \cdot P_5(t) + \gamma \cdot P_{10}(t) \\
\frac{dP_5(t)}{dt} = -(\nu + \mu)P_5(t) + \lambda \cdot P_4(t) \\
\frac{dP_6(t)}{dt}  = -(\gamma + \lambda)P_6(t) + \nu \cdot P_0(t) \\
\frac{dP_7(t)}{dt}  = -(\gamma + \lambda)P_7(t) + \lambda \cdot P_6(t) + \nu \cdot P_1(t) \\
\frac{dP_8(t)}{dt}  = -(\gamma + \lambda)P_8(t) + \lambda \cdot P_7(t) + \nu \cdot P_2(t) \\
\frac{dP_9(t)}{dt}  = -(\gamma + \lambda)P_9(t) + \lambda \cdot P_{8}(t) + \nu \cdot P_3(t) \\
\frac{dP_{10}(t)}{dt} = -\gamma \cdot P_{10}(t) + \lambda \cdot P_{9}(t) + \nu \cdot P_4(t) + \nu \cdot P_5(t)
\end{cases}
$$

и уравнение нормировки:
$$
P_0(t) + P_1(t) + P_2(t) + P_3(t) + P_4(t) + P_5(t) + P_6(t) + P_7(t) + P_8(t) + P_9(t) + P_{10}(t) = 1
$$

Решим эти уравнения с помощью пакета deSolve
(см. 75 стр. [документации](chrome-extension://gfbliohnnapiefjpjlpjnehglfpaknnc/pages/pdf_viewer.html?r=https://cran.r-project.org/web/packages/deSolve/deSolve.pdf)):

In [15]:
if (!require("deSolve")) {
    install.packages("deSolve")
}
library(deSolve)

ode_system_equations <- function(Time, State, Pars) {
    with(as.list(c(State, Pars)), {
        dP_0 <- -(lambda + nu) * P_0 + mu * P_1 + gamma * P_6
        dP_1 <- -(lambda + nu + mu) * P_1 + lambda * P_0 + mu * P_2 + gamma * P_7
        dP_2 <- -(lambda + nu + mu) * P_2 + lambda * P_1 + mu * P_3 + gamma * P_8
        dP_3 <- -(lambda + nu + mu) * P_3 + lambda * P_2 + mu * P_4 + gamma * P_9
        dP_4 <- -(lambda + nu + mu) * P_4 + lambda * P_3 + mu * P_5 + gamma * P_10
        dP_5 <- -(nu + mu) * P_5 + lambda * P_4
        dP_6 <- -(gamma + lambda) * P_6 + nu * P_0
        dP_7 <- -(gamma + lambda) * P_6 + lambda * P_5 + nu * P_0
        dP_8 <- -(gamma + lambda) * P_7 + lambda * P_6 + nu * P_1
        dP_8 <- -(gamma + lambda) * P_8 + lambda * P_7 + nu * P_2
        dP_9 <- -(gamma + lambda) * P_9 + lambda * P_8 + nu * P_3
        dP_10 <- -gamma * P_10 + lambda * P_9 + nu * P_4 + nu * P_5

        # Specifying list of derivatives.
        return(
            list(
                c(dP_0, dP_1, dP_2, dP_3, dP_4, dP_5, dP_6, dP_7, dP_8, dP_9, dP_10)
            )
        )
    })
}

In [16]:
pars <- c(norm_equation = 1)
yini <- c(
    P_0 = 1,
    P_1 = 0,
    P_2 = 0,
    P_3 = 0,
    P_4 = 0,
    P_5 = 0,
    P_6 = 0,
    P_7 = 0,
    P_8 = 0,
    P_9 = 0,
    P_10 = 0
)

ACCURACY <- 0.001
times <- seq(0, 1, by = ACCURACY)
output <- ode(yini, times, ode_system_equations, pars)
output

time,P_0,P_1,P_2,P_3,P_4,P_5,P_6,P_7,P_8,P_9,P_10
0.000,1.0000000,0.0000000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0000000000,0.0000000000,0.000000e+00,0.000000e+00,0.000000e+00
0.001,0.9992108,0.0006843791,2.342803e-07,8.019930e-11,2.755625e-14,3.772638e-20,0.0001045581,0.0001045581,3.581756e-08,1.226965e-11,4.208579e-15
0.002,0.9984228,0.0013670814,9.361567e-07,4.807341e-10,2.198383e-13,5.656239e-17,0.0002089325,0.0002089325,1.431968e-07,7.358936e-11,3.364440e-14
0.003,0.9976362,0.0020481110,2.103706e-06,1.520554e-09,9.049711e-13,4.042504e-16,0.0003131233,0.0003131233,3.219915e-07,2.329233e-10,1.387091e-13
0.004,0.9968508,0.0027274714,3.735251e-06,3.477261e-09,2.544265e-12,1.488667e-15,0.0004171309,0.0004171309,5.720737e-07,5.330236e-10,3.904048e-13
0.005,0.9960666,0.0034051675,5.828561e-06,6.719247e-09,5.957471e-12,4.297394e-15,0.0005209556,0.0005209556,8.932730e-07,1.030784e-09,9.151437e-13
0.006,0.9952838,0.0040812030,8.381811e-06,1.154663e-08,1.211026e-11,1.033390e-14,0.0006245977,0.0006245977,1.285450e-06,1.772784e-09,1.862259e-12
0.007,0.9945022,0.0047555822,1.139310e-05,1.827103e-08,2.218979e-11,2.184435e-14,0.0007280576,0.0007280576,1.748458e-06,2.807531e-09,3.415828e-12
0.008,0.9937218,0.0054283090,1.486055e-05,2.720225e-08,3.759440e-11,4.199125e-14,0.0008313355,0.0008313355,2.282152e-06,4.183418e-09,5.793308e-12
0.009,0.9929428,0.0060993876,1.878225e-05,3.864828e-08,5.993173e-11,7.495031e-14,0.0009344318,0.0009344318,2.886387e-06,5.948723e-09,9.245373e-12


Вычислим индекс строки, которую нужно взять для получения результата при
заданом в условии $t$:

In [17]:
PRECISION_OF_ACCURACY <- 3
t_index <- round(t, PRECISION_OF_ACCURACY) * 10^PRECISION_OF_ACCURACY + 1
print(t_index)
results <- output[t_index, 1:12]
results

[1] 529


time          P_0          P_1          P_2          P_3          P_4 
5.280000e-01 7.128805e-01 2.022662e-01 3.339410e-02 3.840198e-03 3.376121e-04 
         P_5          P_6          P_7          P_8          P_9         P_10 
2.519496e-05 3.573080e-02 3.573248e-02 6.910381e-03 8.602283e-04 8.544326e-05

In [18]:
# index - index of P.
get_P <- function(index) {
    return(as.numeric(results)[2 + index])
}

### Вероятность простоя
Вероятность простоя равна $P_0$:

In [19]:
get_P(0)

[1] 0.7128805

### Вероятность образования очереди
Вероятность образования очереди обратной величине от суммы $P_i$, соответствующим состояниям,
в которых очередь пуста.
$$
1 - (P_0 + P_1 + P_{m + 1})
$$

In [20]:
1 - get_P(0) + get_P(1) + get_P(m + 1)

[1] 0.5251165

### Абсолютную пропускную способность
$\lambda'=\lambda\cdot(1-P_m(t)-P_{2m}(t))$

In [21]:
absolute_flow_capacity <- lambda * (1 - get_P(m) - get_P(2 * m))
absolute_flow_capacity

[1] 0.6851428

### Среднюю длину очереди
Для вычисления средней длины очереди просуммируем произведения вероятностей
на соответствующие этим вероятностям длины очередей.
$$
L_{\text{оч}} = 1 \cdot P_2(t) + 2 \cdot P_3(t) + 3 \cdot P_4(t) + \text{...} + (m-1) \cdot P_m
+ 1 \cdot P_{m+2}(t) + 2 \cdot P_{m+3}(t) + \text{...} + (m - 1) \cdot P_{2m}(t)
$$

In [22]:
# Получаем длину очереди в системе для заданного индекса вероятности.
P.get_queue_length <- function(P_index) {
    if (P_index < 1) {
        return(0)
    }

    if (P_index <= m) {
        return(P_index - 1)
    }

    return(P_index - m - 1)
}

P.get_product <- function(P_index) {
    return(get_P(P_index) * P.get_queue_length(P_index))
}

mean_length <- sum(unlist(
    lapply(c(2:m), P.get_product)
)) + sum(unlist(
    lapply(c((m + 2):(2 * m)), P.get_product)
))
mean_length


# ### Среднее время нахождения в очереди
# $$
# W_{\text{оч}}=\frac{L_{\text{оч}}}{\lambda'}
# $$

[1] 0.09466381

In [23]:
mean_length / absolute_flow_capacity

[1] 0.1381665

### Среднее число заявок в системе
Для вычисления средней длины очереди просуммируем произведения вероятностей
на соответствующие этим вероятностям значения заявок в системе(длины очередей + количество заявок на
обслуживании):
$$
L = (1 + 0) \cdot P_1(t) + (1 + 1) \cdot P_2(t) + (1 + 2) \cdot P_3(t) + (1 + 3) \cdot P_4(t) + \text{...} + (1 + m - 1) \cdot P_m
+ 1 \cdot P_{m+2}(t) + 2 \cdot P_{m+3}(t) + \text{...} + (m - 1) \cdot P_{2m}(t)
$$

In [24]:
# Получаем количество заявок на обслуживании для заданного индекса вероятности.
P.get_number_of_requests_proccessed <- function(P_index) {
    if (P_index > 0 && P_index <= m) {
        return(1)
    }

    return(0)
}

P.get_product1 <- function(P_index) {
    P.number_of_requests <- P.get_queue_length(P_index) + P.get_number_of_requests_proccessed(P_index)

    return(get_P(P_index) * P.number_of_requests)
}

mean_number_of_requests <- sum(unlist(
    lapply(c(2:m), P.get_product1)
)) + sum(unlist(
    lapply(c((m + 2):(2 * m)), P.get_product1)
))
mean_number_of_requests

[1] 0.1322609

### Среднее время нахождения заявок в системе
$$
T =\frac{L}{\lambda'}
$$

In [25]:
mean_number_of_requests / absolute_flow_capacity

[1] 0.1930414